In [7]:
pip install Wand


   ---------------------------------------- 0.0/143.8 kB ? eta -:--:--
   ----------- --------------------------- 41.0/143.8 kB 991.0 kB/s eta 0:00:01
   ------------------------------- -------- 112.6/143.8 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 143.8/143.8 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pytesseract
from PIL import Image
import cv2
import os
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\maxik\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
import re
from pdf2image import convert_from_path
from wand.image import Image as WandImage
from PyPDF2 import PdfReader

from colorama import Fore, Back, Style
import os
import os.path
import numpy as np
import pandas as pd
import fitz  # PyMuPDF
import io
#pytesseract.pytesseract.tesseract_cmd = 'C:\\Users\\maxik\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'

In [10]:
def increase_resolution(input_path, output_path, scale_factor):
    with WandImage(filename=input_path) as img:
        # Set resolution
        img.resolution = (img.width * scale_factor, img.height * scale_factor)
        # Save the high-resolution image
        img.save(filename=output_path)
def pdf_to_image_array(pdf_path, output_image_path, page_number=0):
    output_filename = output_image_path.split(".")
    output_filename = output_filename[0] + "v6" + ".jpg"

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Get the specified page
    page = pdf_document[page_number]

    # Get the pixel data for the page as a bytes object
    image_bytes = page.get_pixmap().samples

    # Close the PDF file
    pdf_document.close()

    # Create a BytesIO object to wrap the bytes data
    image_bytesio = io.BytesIO(image_bytes)
    # Save the image to a file using the BytesIO object
    with open(output_filename, 'wb') as f:
        f.write(image_bytesio.getvalue())
def pytesseractRead(dir):
    output_filename = dir.split(".")
    output_filename = output_filename[0]+"v4"+".jpg"
    increase_resolution(dir, output_filename, 3)
    print(output_filename)
    image = Image.open(output_filename)  # Replace with the path to your image
    text_colums = pytesseract.image_to_string(image)
    text_colums_inline = ""
    for i in text_colums:
        if(i == '\n'):
            text_colums_inline=text_colums_inline+','
        else:
            text_colums_inline=text_colums_inline+i
    custom_config = r'--psm 6'  # Horizontal text mode
    text_lines = pytesseract.image_to_string(image, config=custom_config)
    print(text_lines)
    #text processing begins here
    #text_colums_inline
    

    print(text_lines)
    return text_colums
def crop_item(dir, Le_Up_Ri_Lo, root_dir):
    try:
        image = Image.open(dir)  # Replace with the path to your image
    except:
        print(dir+" erro failed to read")
        return 1
    # Define the coordinates of the rectangle you want to crop (left, upper, right, lower)
    left = 0
    upper = 100
    right = 3700
    lower = 4900
    #^ nuron network to identify cordinates of a data frame rectangle, 
    # Crop the image
    cropped_image = image.crop((left, upper, right, lower))

    # Save the cropped image
    list_filename = dir.split('.')
    filename=list_filename[0]+".png"
    cropped_image.save(filename)
    return filename
def itr_dir(dir, extension, jdg_dir=r"C:\Users\maxik\Documents\Github\Nuron-text-recognition\WNB-PUCD6-DH-1432-jdg"):  # Replace with your desired file extension
    # List all files in the directory

    # Filter files by the specified extension
    # Print the filtered file list
    try: 
        print("hello")
        file_list = os.listdir(jdg_dir)
        filtered_files = [file for file in file_list if file.endswith(".jpg")]
        print("hello!")
        for file in filtered_files:
            dir_fle = ""
            print("hello")
            dir_file = jdg_dir+'\\'+file
            list_filenames = file.split('.')
            print('extrancting')
            output_dir = crop_item(dir_file, 0, jdg_dir)
            if(output_dir != 1):
                print("crop")
                print(output_dir)
                text_output = pytesseractRead(output_dir)
            print(text_output)
            break
    except:
        print("hello")
        file_list = os.listdir(dir)
        filtered_files = [file for file in file_list if file.endswith(extension)]
        os.makedirs(jdg_dir, exist_ok=True)
        for file in filtered_files:
            dir_file = ""
            dir_file = dir+'\\'+file
            list_filenames = file.split('.')
            if(list_filenames[1] == 'pdf'):
                    #pdf_to_jpg(dir_file, dir)
                    #extract_text_from_column(dir_file, 0)
                pdf_to_image_array(file, r"C:\Users\maxik\Documents\Github\Nuron-text-recognition\WNB-PUCD6-DH-1432-jdg")
                #print(dir_file)
input_dir=r"C:\Users\maxik\Documents\GitHub\Nuron-text-recognition\WNB-PUCD6-DH-1432"
jdg_dir = r"C:\Users\maxik\Documents\Github\Nuron-text-recognition\WNB-PUCD6-DH-1432-jdg"
itr_dir(input_dir, ".pdf")

hello
hello!


In [1]:
import fitz  # PyMuPDF
import numpy as np
from PIL import Image

def pdf_to_image_array(pdf_path, page_number=0, output_image_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Get the specified page
    page = pdf_document[page_number]

    # Get the pixel data for the page as a NumPy array
    image_array = page.get_pixmap().samples

    # Close the PDF file
    pdf_document.close()
    image = Image.fromarray(image_array)

    # Save the image to a file
    image.save(output_image_path)

def save_image_array_as_image(image_array, output_image_path):
    # Convert the NumPy array to a PIL Image
    image = Image.fromarray(image_array)

    # Save the image to a file
    image.save(output_image_path)

# Example usage
pdf_path = 'path/to/your/file.pdf'
output_image_path = 'output_image.png'

# Convert PDF to NumPy array
image_array = pdf_to_image_array(pdf_path)

# Save NumPy array as an image
save_image_array_as_image(image_array, output_image_path)


FileNotFoundError: no such file: 'path/to/your/file.pdf'